In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import ROM
import lal, lalsimulation
from itertools import combinations 

In [2]:
def generate_waveforms(
duration,            # Length in geometric units to truncate the waveform to
mass_1,              # Mass of primary in solar masses
mass_2,              # Mass of secondary in solar masses
l_max,
modes,
                     # To generate a waveform model in the time domain, we need to specify the binary parameters
f_min        = 20.,  # Starting frequency
f_ref        = 20.,  # Reference frequency
delta_t      = 1/2048,# Sampling time (e.g. 1/2048 Hz)
phase        = 0.0,  # Phase at reference frequency
                     # Dimensionful spin, S_i = m^2_i * \chi_i 
                     # Frame is defined such that \hat{L} is pointing along \hat{z}
chi_1x       = 0,    # Dimensionless spin of primary in x-direction
chi_1y       = 0,    # Dimensionless spin of primary in y-direction
chi_1z       = 0,    # Dimensionless spin of primary in z-direction
chi_2x       = 0,    # Dimensionless spin of secondary in x-direction
chi_2y       = 0,    # Dimensionless spin of secondary in y-direction
chi_2z       = 0,    # Dimensionless spin of secondary in z-direction
inclination  = 0.5,  # Inclination of L (orbital ang. mom.) with respect to J (total ang. mom.)
distance     = 100 * 1e6 * lal.PC_SI, # Luminosity distance in SI units
longAscNodes = 0,
meanPerAno   = 0,
eccentricity = 0,
laldict      = lal.CreateDict(), # LAL Dictionary (allows us to pass additional flags and options to the waveform generator
verbose      = False
):
    hlm         = lalsimulation.SimInspiralChooseTDModes(phase,delta_t,mass_1,
                                                    mass_2,chi_1x, chi_1y, chi_1z,
                                                    chi_2x, chi_2y, chi_2z,
                                                    f_min,f_ref, distance, laldict, l_max, 
                                                    lalsimulation.IMRPhenomTHM
                                                )
    
    time_array = lalsimulation.SphHarmTimeSeriesGetMode(hlm, 2, 2).deltaT * np.arange(len(lalsimulation.SphHarmTimeSeriesGetMode(hlm, 2, 2).data.data))
    hlms = lalsimulation.SphHarmTimeSeriesGetMode(hlm, 2, 2).data.data

    # Sets the arrays such that the peak amplitudes line up time-wise
    strain = np.zeros(len(hlms))
    strain += np.abs(hlms)**2
    t_max = time_array[np.argmax(strain)]
    time_array -= t_max

    # Geometric unit conversion and waveform truncation
    mass_total = (mass_1 + mass_2)/lal.MSUN_SI
    mass_ratio = mass_1/mass_2 
    time_geom = time_array/(mass_total * lal.MTSUN_SI)
    hlms_geom = hlms * distance / (mass_total * lal.MRSUN_SI)
    
    waveform_length = duration * mass_total * lal.MTSUN_SI
    array_length = int(np.floor(waveform_length/delta_t))

    time_trunc = time_geom[-array_length:]
    hlms_trunc = hlms_geom[-array_length:]

    return time_trunc, hlms_trunc

In [3]:
# Define some parameters regarding the masses. q is a mass ratio (m1/m2)
q = np.arange(1, 10, 0.1)
mass_total = 50 * lal.MSUN_SI
m_1 = (mass_total*q)/(q+1)
m_2 = mass_total/(q+1)
l_max = 2
modes = [(l,m) for l in range(2,l_max+1) for m in range(-l,l+1)]
distance = 100 * 1e6 * lal.PC_SI
delta_t = 1/2048

# Waveforms can take many modes. E.g the 2,2 mode is obtained via waveforms[x][(2,2)]
data = [generate_waveforms(5000, m_1[i],m_2[i], l_max, modes, distance=distance, delta_t=delta_t) for i in range(len(q))]
times, waveforms = zip(*data)
mode_22 = np.asarray([waveforms[i] for i in range(len(waveforms))])

In [6]:
from pycbc.filter import match
from pycbc import types, fft, waveform, psd

# Mismatch
h_plus_A = mode_22[5]
h_plus_B = mode_22[6]

hpA = types.TimeSeries(h_plus_A,delta_t=delta_t)
hpB = types.TimeSeries(h_plus_B,delta_t=delta_t)

# Resize the waveforms to the same length
tlen = max(len(hpA), len(hpB))

# Generate the aLIGO ZDHP PSD
delta_f   = 1.0 / hpA.duration
f_len     = tlen//2 + 1
f_low     = 20.0 
det_psd   = psd.aLIGOLateHighSensitivityP1200087(f_len, delta_f, f_low)
     

# Note: This takes a while the first time as an FFT plan is generated
# subsequent calls are much faster.
m, i = match(hpA, hpB, psd=det_psd, low_frequency_cutoff=f_low)
print(m)

ValueError: For C2C FFT, len(outvec) must be nbatch*size